In [4]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\OrbitalNeuralMethods`


In [5]:
n = 2
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 10
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

rhf = RHF(system);
rhf = compute_ground_state!(rhf, max_iter = 100, tol = 1e-10, verbose=0);
slater = Slater(rhf);

In [6]:
function energy_opt(wf, optimizer)
    ham = HOCoulomb(0.25, shielding = 0.25);

    metro = Importance(equils = 1e4, samples = 1e4, step = 0.01);
    t = @elapsed wf_final, wf_opt, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);
    println("\nGradient time: $(t) s")
    return wf_final, wf_opt
end

function energy_block(wf)
    samples_b = 2^23
    metro_b = Importance(equils = 1e6, samples = samples_b, step = 0.01);
    t = @elapsed result = blocking(wf, ham, metro_b, nthreads = 12);
    digits = Integer(-floor(log10(result.E_err)))
    print("$(round(result.E, digits=digits)) ± $(round(result.E_err, digits=digits)), $(round((result.E - 0.8247)/0.8247, digits=digits))")
    println("\nBlocking time: $(t) s")
end

energy_block (generic function with 1 method)

### 8

In [7]:
nn = NeuralNetwork([Dense(8), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 500, tol = 1e-6);

wf8_final, wf8_opt = energy_opt(wf, optimizer);
energy_block(wf8_final);
energy_block(wf8_opt);

E = 0.830693 iter = 500/500                                      
Gradient time: 23.3990929 s
0.8314 ± 0.0003, 0.0082
Blocking time: 6.3939591 s
0.883 ± 0.002, 0.071
Blocking time: 5.3279222 s


### 16

In [8]:
nn = NeuralNetwork([Dense(16), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 500, tol = 1e-6);

wf16_final, wf16_opt = energy_opt(wf, optimizer);
energy_block(wf16_final);
energy_block(wf16_opt);

E = 0.828267 iter = 500/500                                      
Gradient time: 19.0896283 s
0.8275 ± 0.0002, 0.0034
Blocking time: 6.3423507 s
0.862 ± 0.001, 0.045
Blocking time: 6.1381013 s


### 32

In [9]:
nn = NeuralNetwork([Dense(32), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 500, tol = 1e-6);
wf32_final, wf32_opt = energy_opt(wf, optimizer);
energy_block(wf32_final);
energy_block(wf32_opt);

E = 0.82818 iter = 500/500                                       
Gradient time: 23.5188022 s
0.8317 ± 0.0002, 0.0084
Blocking time: 8.165799 s
0.849 ± 0.002, 0.029
Blocking time: 7.9073383 s


In [10]:
samples_b = 2^27
metro_b = Importance(equils = 1e6, samples = samples_b, step = 0.01);
@time result = blocking(wf32_final, ham, metro_b, nthreads = 12);
digits = Integer(-floor(log10(result.E_err)))
print("$(round(result.E, digits=digits)) ± $(round(result.E_err, digits=digits)), $(round((result.E - 0.8247)/0.8247, digits=digits))")

 81.676279 seconds (2.43 k allocations: 11.001 GiB, 2.88% gc time)
0.83157 ± 4.0e-5, 0.00833

### 48

In [14]:
nn = NeuralNetwork([Dense(48), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 500, tol = 1e-6);

wf48_final, wf48_opt = energy_opt(wf, optimizer);
energy_block(wf48_final);
energy_block(wf48_opt);

E = 0.829947 iter = 500/500                                      
Gradient time: 28.8671152 s
0.8295 ± 0.0001, 0.0059
Blocking time: 10.1108966 s
0.837 ± 0.001, 0.015
Blocking time: 9.6467209 s


### 32 + 8

In [13]:
nn = NeuralNetwork([Dense(32), Tanh, Dense(8), Tanh, Dense(1), Exp], n);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 500, tol = 1e-6);

wf328_final, wf328_opt = energy_opt(wf, optimizer);
energy_block(wf328_final);
energy_block(wf328_opt);

E = 0.864385 iter = 500/500                                      
Gradient time: 32.3131536 s
0.851 ± 0.0003, 0.0319
Blocking time: 11.5121472 s
0.868 ± 0.001, 0.053
Blocking time: 11.300671 s
